# Esercizio 2 - Content2Word

L'obiettivo di questo esercizio è, date le definizioni presenti nel file *def.csv* usate per l'esercizio 1, risalire al termine che le descrive. Si tratta quindi di una ***ricerca onomasiologica***.

### Approccio

- Prendiamo gli *n* termini più frequenti nelle nostre definizioni, essi saranno i ***genus*** utilizzati per restringere la ricerca su *WordNet*.

- ***Fase di text cleaning***: rimozione di *stopwords* e *lemmatizzazione*.
- Recupere tutto il sottoalbero di iponimi dei genus e salvataggio delle loro definizioni (glosse).
- Confronto tra le definizioni di *Wordnet* e le *m* parole più frequenti delle nostre definizioni. 
- Il synset restituito è quello che massimizza l'*overlap* tra la definizione di *Wordnet* e la nostra lista di parole.

In [7]:
from nltk.corpus import stopwords
from collections import Counter
from gensim.test.utils import simple_preprocess
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.corpus import wordnet as wn
import random

### Funzioni

In [6]:
def get_text_from_file(path): # rimuove le stopwords e restituisce lista di parole nel file in path
    file = []
    stop_words = set(stopwords.words('english'))
    with open (path, 'r') as f:
        for row in f:
            filtered_s = [w for w in word_tokenize(row) if not w.lower() in stop_words]
            file.append(simple_preprocess(str(filtered_s), deacc=True))
    f.close()
    return file

def get_most_freq_words(text, nword): # restituisce le n parole più frequenti in text, per ogni riga del documento
    genus = []
    for row in text:
        c = Counter()
        c.update(row)
        genus.append(c.most_common(nword))
    return genus

def get_hypos(word): # restituisce tutti gli iponimi di word
    syn = get_synset(word)
    hypo_list = []
    if(syn is not None):
        hypo_list = list(set([w for s in syn.closure(lambda s:s.hyponyms()) for w in s.lemma_names()]))
    return hypo_list

def get_hypers(word): # restituisce tutti gli iperonimi di word
    syn = get_synset(word)
    hyper_list = []
    if(syn is not None):
        hyper_list = list(set([w for s in syn.closure(lambda s:s.hypermyms()) for w in s.lemma_names()]))
    return hyper_list

def get_synset(word): # restituisce il primo synset di word
    if(len(wn.synsets(word)) > 0):
        return wn.synsets(word)[0]
    return None


### Esecuzione

In [5]:
num_genus = 3 
num_most_freq_word = 10 
starting_words = ["Emotion", "Person", "Revenge", "Brick"]

In [8]:
file = get_text_from_file('../data/def.csv')
genus = get_most_freq_words(file, num_genus)

In [9]:
key_words_defs = get_most_freq_words(file, num_most_freq_word) # 10 parole più frequenti per ogni riga del documento

for i in range(len(genus)):
    
    # collasso key_words_defs in un unico array
    key_row = []
    for el in key_words_defs[i]:
        key_row.append(el[0])

    # salvo gli iponimi di ogni genus
    hypo_list, hypo_def = [], []
    for el in genus[i]:
        hypo_list.append(get_hypos(el[0])) 
     
    hypo_list = [x for xs in hypo_list for x in xs]

    # definizioni di ogni iponimo
    for hypo in hypo_list:
        hypo_def.append((hypo, get_synset(hypo).definition()))
    
    # salvo parole che massimizzano l'overlap e i punteggi (numero di intersezioni)
    res = []
    for wndef in hypo_def: # definizioni di ogni iponimo 
        score = 0
        imp_words = []
        for key_word in key_row: # definizioni in def.csv
            if(key_word in wndef[1]):
                score += 1
                imp_words.append(key_word)      

        res.append((score, wndef[0], imp_words, wndef[1]))
        
    # ordino sullo score e inverto
    sorted_list = sorted(res, key=lambda x: x[0]) 
    sorted_res = list(reversed(sorted_list))
    
    print("\nOriginal Word:", starting_words[i], "/  Genus -->",genus[i][0])
    for k in range(min(len(sorted_res), 5)):
        print(f'Word: *{sorted_res[k][1]}*, score: *{sorted_res[k][0]}*')

/home/fazza/.local/lib/python3.8/site-packages/nltk/corpus/reader/wordnet.py:580: UserWarning: Discarded redundant search for Synset('gusto.n.01') at depth 3
  for synset in acyclic_breadth_first(self, rel, depth):
/home/fazza/.local/lib/python3.8/site-packages/nltk/corpus/reader/wordnet.py:580: UserWarning: Discarded redundant search for Synset('antagonism.n.03') at depth 4
  for synset in acyclic_breadth_first(self, rel, depth):



Original Word: Emotion /  Genus --> ('feeling', 11)
Word: *comfort*, score: *3*
Word: *painfulness*, score: *3*
Word: *hatred*, score: *3*
Word: *astonishment*, score: *3*
Word: *appetence*, score: *3*


/home/fazza/.local/lib/python3.8/site-packages/nltk/corpus/reader/wordnet.py:580: UserWarning: Discarded redundant search for Synset('amerindian.n.01') at depth 2
  for synset in acyclic_breadth_first(self, rel, depth):
/home/fazza/.local/lib/python3.8/site-packages/nltk/corpus/reader/wordnet.py:580: UserWarning: Discarded redundant search for Synset('black.n.05') at depth 2
  for synset in acyclic_breadth_first(self, rel, depth):
/home/fazza/.local/lib/python3.8/site-packages/nltk/corpus/reader/wordnet.py:580: UserWarning: Discarded redundant search for Synset('black_woman.n.01') at depth 3
  for synset in acyclic_breadth_first(self, rel, depth):
/home/fazza/.local/lib/python3.8/site-packages/nltk/corpus/reader/wordnet.py:580: UserWarning: Discarded redundant search for Synset('eurafrican.n.01') at depth 3
  for synset in acyclic_breadth_first(self, rel, depth):
/home/fazza/.local/lib/python3.8/site-packages/nltk/corpus/reader/wordnet.py:580: UserWarning: Discarded redundant search fo


Original Word: Person /  Genus --> ('human', 26)
Word: *Brahui*, score: *2*
Word: *Anglo-Indian*, score: *2*
Word: *Turkmen*, score: *2*
Word: *Cheremiss*, score: *2*
Word: *fellow_member*, score: *2*

Original Word: Revenge /  Genus --> ('someone', 14)
Word: *sounding_board*, score: *5*
Word: *hatred*, score: *4*
Word: *stolidity*, score: *4*
Word: *impassiveness*, score: *4*
Word: *hate*, score: *4*


/home/fazza/.local/lib/python3.8/site-packages/nltk/corpus/reader/wordnet.py:580: UserWarning: Discarded redundant search for Synset('iceberg.n.01') at depth 3
  for synset in acyclic_breadth_first(self, rel, depth):
/home/fazza/.local/lib/python3.8/site-packages/nltk/corpus/reader/wordnet.py:580: UserWarning: Discarded redundant search for Synset('piece_of_cloth.n.01') at depth 4
  for synset in acyclic_breadth_first(self, rel, depth):
/home/fazza/.local/lib/python3.8/site-packages/nltk/corpus/reader/wordnet.py:580: UserWarning: Discarded redundant search for Synset('archeological_remains.n.01') at depth 5
  for synset in acyclic_breadth_first(self, rel, depth):
/home/fazza/.local/lib/python3.8/site-packages/nltk/corpus/reader/wordnet.py:580: UserWarning: Discarded redundant search for Synset('article_of_commerce.n.01') at depth 5
  for synset in acyclic_breadth_first(self, rel, depth):
/home/fazza/.local/lib/python3.8/site-packages/nltk/corpus/reader/wordnet.py:580: UserWarning: Disc


Original Word: Brick /  Genus --> ('used', 22)
Word: *brick*, score: *5*
Word: *building_material*, score: *4*
Word: *kaolin*, score: *3*
Word: *shingle*, score: *3*
Word: *manure*, score: *3*


## Analisi dei risultati

Alcuni problemi riscontrati:

- **I genus**: i genus di riferimento hanno poco a che vedere con i termini, come ad esempio *"someone"* per *revenge*. Questo è portato dal dataset in input. Si potrebbe ripulire il dataset rimuovendo le parole che non hanno a che vedere con il termine originale, ma così facendo si andrebbe a compromettere l'esercizio e si renderebbe questo approccio poco scalabile su altre basi di dati.
   
- **Iponimi**: non è detto che il termine ricercato sia un iponimo dei genus che selezioanti, infatti potrebbe essere 
   un iperonimo o essere proprio in un punto completamente diverso dell'albero di *Wordnet*. Un possibile miglioramento dell'algoritmo potrebbe
   essere quello di andare a prelevare altri synset oltre agli iponimi del genus, ad esempio andando a prelevare anche i fratelli del genus,
   senza allontarsi troppo per non far esplodere la complessità dell'algoritmo.

- **Funzione di similarità**: l'algoritmo è basato su uno *score* che corrisponde a quante parole simili ci sono nelle definizioni. Si potrebbe 
   incrementare lo *score* sulla base di altri fattori, come la funzione di similarità di wordnet *path_similarity*.
